In [1]:
USE_FULL_1900_DIM_MODEL = False # if True use 1900 dimensional model, else use 64 dimensional one.

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set seeds
tf.set_random_seed(42)
np.random.seed(42)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

In [3]:
batch_size = 50
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# Before you can train your model, 
sequences = []
with open("clinical_vl_1.txt", "r") as source:
    with open("formatted.txt", "w") as destination:
        for i,seq in enumerate(source):
            seq = seq.strip()
            sequences.append(seq)
            if b.is_valid_seq(seq) and len(seq) < 275: 
                formatted = ",".join(map(str,b.format_seq(seq)))
                destination.write(formatted)
                destination.write('\n')

In [5]:
## 
average_hidden_list = []
final_hidden_list = []
hs_list = []
final_cell_list = []


num2 = range(0, 50)
x = 0
y = 50
for i in num2:
    num1 = range(x, y)
    for j in num1:
        avg_hidden, final_hidden, final_cell, hs_out = (b.get_rep_hs(sequences[j]))
        average_hidden_list.append(avg_hidden)
        final_hidden_list.append(final_hidden)
        final_cell_list.append(final_cell)
        hs_list.append(hs_out)
    print('rep')
    x = x + 50
    y = y + 50
    


rep
rep
rep
rep
rep
rep
rep
rep
rep
rep
rep
rep


IndexError: list index out of range

In [6]:
average_hidden_pd = pd.DataFrame(np.row_stack(average_hidden_list))
final_hidden_pd = pd.DataFrame(np.row_stack(final_hidden_list))
hidden_state = pd.DataFrame(np.row_stack(hs_list))
print(hidden_state)

             0         1         2         3         4         5         6   \
0     -0.165004  0.367741  0.358379 -0.652705  0.310745  0.112354  0.319343   
1     -0.142708  0.018597 -0.007884 -0.945147 -0.164700 -0.158298  0.149896   
2     -0.105781  0.035168 -0.087267 -0.982521 -0.109088 -0.101870  0.118040   
3     -0.135685  0.049581 -0.129095 -0.984502 -0.079332 -0.069060  0.048985   
4     -0.081012  0.038746 -0.128814 -0.977766 -0.046916 -0.078257  0.032707   
5     -0.053119  0.077448 -0.137827 -0.973507 -0.046264 -0.121132  0.060475   
6     -0.058201  0.029613 -0.116138 -0.981766 -0.054295 -0.123863  0.028536   
7     -0.032765  0.057504 -0.106545 -0.984190 -0.045189 -0.151356  0.042968   
8     -0.016855  0.064332 -0.083712 -0.993859 -0.033166 -0.219582  0.028394   
9      0.033697  0.029008 -0.081737 -0.992128 -0.042845 -0.264039  0.007066   
10     0.054237 -0.029784 -0.076597 -0.994180 -0.026868 -0.362712  0.004869   
11     0.021025 -0.000586 -0.065509 -0.993353 -0.051

In [7]:
average_hidden_pd.to_csv("clinical_vl_1_reps.csv")
final_hidden_pd.to_csv("clinical_vl_1_finalhidden.csv")

In [8]:
import pickle
save_loc = "C:\\Users\\pkinn\\Documents\\UniRep\\full representations\\emi larger set\\"
data_name = 'clinical_vl_1'
file_append = '.pickle'


fn = save_loc + data_name + 'avg_hidden' + file_append
with open(fn, 'wb') as f:
    pickle.dump(average_hidden_list, f)

fn = save_loc + data_name + 'final_hidden' + file_append
with open(fn, 'wb') as f:
    pickle.dump(final_hidden_list, f)

fn = save_loc + data_name + 'final_cell' + file_append
with open(fn, 'wb') as f:
    pickle.dump(final_cell_list, f)
   
fn = save_loc + data_name + 'hidden_state' + file_append
with open(fn, 'wb') as f:
    pickle.dump(hs_list, f)
   
fn = save_loc + data_name + 'all_output_hs' + file_append
with open(fn, 'wb') as f:
    pickle.dump([average_hidden_list, final_hidden_list, final_cell_list, hs_list], f)